# Test-aankoop - Modeling

In [ ]:
#!pip install lightgbm

In [ ]:
# development

In [415]:
import pandas as pd
from datetime import datetime
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [416]:
# read the prepared data from the prepared data script
# Read the sales table
preparedSales = pd.read_csv("C:/Users/ccui/Box Sync/SO Delivery_Marmo/Marmo_OPEX2020/Planning Marmo/2 Working Documents/data/Data Voor El 61/salesToModel.csv",
                            sep=';',
                            header = 'infer'
                           )
preparedSales['date']= pd.to_datetime(preparedSales['date']) 
  
preparedSales.head()

,artikelnr,date,klant,besteldvolume,branche,year,month,week,day,weekday,...,week_before_herfst,week_before_kerst,Wind,Humidity,CloudCover,TemperatureMin,TemperatureMax,PrecipProbability,Icon,promoType
0,11001000000,2018-10-26,60344,0.64,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1
1,11001000000,2018-10-26,60381,0.64,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1
2,11001000000,2018-10-26,60391,1.28,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1
3,11001000000,2018-10-26,60432,0.64,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1
4,11001000000,2018-10-26,60449,0.64,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1


In [417]:
# select the product & store

selectedProduct = 1100100032002 # 1100100032002
selectedStore = 6403 # 6401
#baseTable = preparedSales[(preparedSales["artikelnr"]==1100100032002) & (preparedSales["klant"]==6401)]
baseTable = preparedSales[(preparedSales["artikelnr"]==selectedProduct)]
baseTable = baseTable[(baseTable["klant"]==selectedStore)]

baseTable.head()

,artikelnr,date,klant,besteldvolume,branche,year,month,week,day,weekday,...,week_before_herfst,week_before_kerst,Wind,Humidity,CloudCover,TemperatureMin,TemperatureMax,PrecipProbability,Icon,promoType


In [418]:
# select the time periods for train & test
df_train = baseTable[(baseTable["date"]>=datetime(2017, 1, 1)) & (baseTable["date"]<=datetime(2019, 12, 31))]
print( "training set date range: " + str(min(df_train['date']).date()) + " to " + str(max(df_train['date']).date()))

df_test = baseTable[(baseTable["date"]>=datetime(2020, 1, 1)) & (baseTable["date"]<=datetime(2020, 4, 30))]
print( "test set date range: " + str(min(df_test['date']).date()) + " to " + str(max(df_test['date']).date()))

ValueError: min() arg is an empty sequence

In [ ]:
# set the date column as index column
df_train = df_train.set_index(['date', 'artikelnr', 'klant'])
df_test = df_test.set_index(['date', 'artikelnr', 'klant'])

df_train.head()

In [ ]:
# set the dependent variable (Y) & drop it from the independent variables (X) 
y_train = df_train['besteldvolume']
y_test = df_test['besteldvolume']

X_train = df_train.drop('besteldvolume', axis=1)
X_test = df_test.drop('besteldvolume', axis=1)

In [ ]:
#df_train_null['pred'], df_eval_null['pred'], df_test_null['pred'] = 0, 0, 0
#df_train_null['error'], df_eval_null['error'], df_test_null['error'] = 0, 0, 0

In [ ]:
features = X_train.columns
categorical_features = list(features.drop(['year', 'Wind', 'Humidity', 'CloudCover', 'TemperatureMin', 'TemperatureMax', 'PrecipProbability']))
print(categorical_features)

In [ ]:
df_train.head()

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(data=X_train, label=y_train, categorical_feature=categorical_features)
lgb_test = lgb.Dataset(data=X_test, label=y_test, categorical_feature=categorical_features, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 7,
    'learning_rate': 0.001,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.8,
    'bagging_freq': 10,
    'max_depth': 5,
    'verbose': 50,
    'early_stopping_round': 20,
}

print('Starting training...')
# train
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=10_000,
    valid_sets=[lgb_train],
#     valid_sets=lgb_test
)

In [ ]:
featureNames = gbm.feature_name()
featureImportances = gbm.feature_importance()

importances = {'Feature':featureNames,'Importance':featureImportances}
featureImportanceTable = pd.DataFrame(importances)

featureImportanceTable['artikelnr'] = selectedProduct
featureImportanceTable['klant'] = selectedStore
featureImportanceTable.head()

In [ ]:
df_train['pred'] = gbm.predict(X_train, num_iteration=gbm.best_iteration)
df_train['error'] = df_train['besteldvolume'] - df_train['pred']
print('The rmse for the training set is:', mean_squared_error(y_train, df_train['pred']) ** 0.5)

In [ ]:
df_test['pred'] = gbm.predict(X_test, num_iteration=gbm.best_iteration)
df_test['error'] = df_test['besteldvolume'] - df_test['pred']
print('The rmse for the test set is:', mean_squared_error(y_test, df_test['pred']) ** 0.5)

In [ ]:
df_result = df_test[['besteldvolume', 'pred', 'error']]
print(df_result.reset_index().head())

In [ ]:
# Plot feature importances
ax = lgb.plot_importance(gbm, max_num_features=20)
plt.show()

In [ ]:
df_pred_actual = df_test.reset_index()
df_pred_actual = df_pred_actual.set_index('date')
df_pred_actual.head()

In [ ]:
df_plot = df_pred_actual.sort_index()
fig1 = plt.figure(figsize=(16,6))
plt.plot(df_plot.index, df_plot['besteldvolume'], label='actuals')
plt.plot(df_plot.index, df_plot['pred'], label='prediction')
plt.ylim(0, 10)
plt.legend()